In [2]:
# automatically update imports
%load_ext autoreload
%autoreload 2

# For graphs
%matplotlib inline

# snippet for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import extract_uses.parameters as parameters
import extract_uses.utils as utils

# City to download: Grenoble
city_country = "grenoble_france"#"grenoble_france"
citiesFolder = 'cities/'
# Population count file
popu_count_file = "/home/lgervasoni/gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-2000/gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals_2000.tif"

# Get the required cities shapefile
utils.getCityShapefile(citiesFolder, [ city_country ])

# Set input files
parameters.setInputFiles(citiesFolder, city_country, popu_count_file, numberOfCuts = 4)
#parameters.setInputFiles(citiesFolder, city_country, popu_count_file = None, numberOfCuts = 4)

('Existing shapefile for', 'grenoble_france')


In [5]:
import extract_uses.main

extract_uses.main.process()

('Folder already exists. Assumption: Already processed. Only mapping categories and exiting.', 'cities/grenoble_france/')
Complete processing: --- 0.403701035182 minutes ---


In [ ]:
import loaders
import extract_uses.utils as utils

uses_shp = citiesFolder+city_country+"/full_uses.shp"
bbox = utils.getBoundingBox(uses_shp)

geo_graph = loaders.load_graph(city_country, bbox)
pois_df = loaders.load_pois(city_country, uses_shp)
centrality_df = loaders.load_graph_centrality(city_country, geo_graph)
kde_df = loaders.load_graph_kde(city_country, geo_graph, pois_df)

# Grid KDE
categories_kde_dict = loaders.load_grid_kde(city_ref, all_pois_df)

('Bounding box:', [45.113017726416615, 5.573125356566398, 45.243980680751875, 5.9029513504478865])
Querying locally for `['nodes', 'edges']`
`['nodes', 'edges']` is/are not stored locally. Determining it/them through `graph_from_bbox` method
Saving data for `['nodes', 'edges']` at `/home/lgervasoni/urbansprawl/urban_analysis/hdfs_store/grenoble_france.h5`
The data has been stored locally with success
Querying locally for `['pois']`
`['pois']` is/are not stored locally. Determining it/them through `get_extracted_osm_points` method
Saving data for `['pois']` at `/home/lgervasoni/urbansprawl/urban_analysis/hdfs_store/grenoble_france.h5`
The data has been stored locally with success
Querying locally for `['graph_centrality']`
`['graph_centrality']` is/are not stored locally. Determining it/them through `get_centrality_df` method


In [6]:
import kde

categories_kde = kde.get_categories_kde(pois_df, bbox)

In [118]:
categories_kde.keys()

[u'shop', u'residential', u'commercial/industrial', u'leisure/amenity']

In [106]:
keys = list(map(lambda cat : 'kde_' + str(cat), categories_kde.keys()))

In [108]:
dfs = list(map(pd.DataFrame, categories_kde.values()))

In [77]:
ser = pd.Series(categories_kde.values(), index=list(map(str, categories_kde.keys())))

In [116]:
print('Keys:',keys)
print('Dfs:',dfs)

with pd.HDFStore(loaders._generate_file_path(city_ref), 'a') as store:
    for key, df in zip(keys, dfs):
        store.put(key, df, format='table')

In [19]:
import pickle
# TODO: create a `loaders` method for `category_kde`
with open('categories_kde.p', 'wb') as f:
    pickle.dump(categories_kde, f)

In [49]:
import numpy as np

phi = 0
categories_list = categories_kde.items()
# let: n be the total number of categories; l be the total number of grid (lattice) points
n = len(categories_list[0][0])
l = len(categories_list[0][1])**2 # TODO: more `correct` GRIDSIZE handling
# [[phi += np.sum(vi - vj) for j, (kj, vj) in enumerate(categories_list[i:])] for i, (ki, vi) in enumerate(categories_list)]

for i, (ki, vi) in enumerate(categories_list):
    for j, (kj, vj) in enumerate(categories_list[i:]):
        phi += np.sum(vi - vj) # here i would divide it by l, but computationally i can do that at the end

phi_mix = phi / l # (l * n * (n-1) * .5)        

In [50]:
phi_mix

1.1384253034912237

In [53]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib qt
n_rows = len(categories_kde)
base_figsize = 8
xx, yy = kde._grid_from_bbox(bbox)

fig = plt.figure(figsize=(8, 8 * n_rows))
i = 1
for label, z in categories_kde.items():
    category_pois = all_pois_df[all_pois_df['key'] == label]
    ax1 = fig.add_subplot(n_rows, 2, i)
    ax2 = fig.add_subplot(n_rows, 2, i+1, projection='3d')
    ax1.scatter(category_pois['lon'], category_pois['lat'])
    ax1.set_ylabel(label, rotation=90)
    ax2.plot_surface(xx, yy, z, cmap=cm.RdYlBu)
    i += 2
    ax2.set_xlabel('lon', labelpad=-10)
    ax2.set_ylabel('lat', labelpad=-10)
    ax2.zaxis.set_rotate_label(False) # workaround
    ax2.set_zlabel(r'$f_c$', rotation=0, labelpad=-15)
    ax2.w_xaxis.set_ticklabels([])
    ax2.w_yaxis.set_ticklabels([])
    ax2.w_zaxis.set_ticklabels([])
    
plt.show()